# 1. <span style='color:green'>Zaimportowanie potrzebnych bibliotek oraz danych z pliku .csv

In [0]:
import pandas as pd
import numpy as np
import random as rd

iris = pd.read_csv('iris.csv')
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


# 2. <span style='color:green'>Implementacja klasy służącej do przygotwowywania danych do klasyfikacji

In [0]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j = rd.randint(0,i)
            x.iloc[i], x.iloc[j] = x.iloc[j], x.iloc[i]
        return x

    @staticmethod
    def split_set(x):
        k = int(len(x)*0.7)
        train = x[:k]
        val = x[k:]
        return train, val
    
    @staticmethod
    def normalize(x):
        val = x.select_dtypes(exclude="object")
        columns = val.columns.tolist()
        for c in columns:
            current_set = x.loc[:, c]
            maximum, minimum = max(current_set), min(current_set)
            for r in range(0,len(x),1):
                value = (x.at[r, c] - minimum) / (maximum - minimum)
                x.at[r, c] = value

# 3. <span style='color:green'>Implementacja klasy służacej do klasyfikacji naiwnym klasyfikatorem Bayesa

*Zrezygnowałem z pisania własnych metod wyliczających średnią oraz odchylenie standardowe na rzecz tych wbudowanych w Pandas*<br><br>
Średnia artmetyczna: $\bar{x} = \frac{x_1+x_2+\cdots+x_n}{n}$ <br><br>
Odchylenie standardowe: $\sigma = \sqrt{\frac{(x_1-\bar{X})^2+(x_2-\bar{X})^2+\cdots+(x_n-\bar{X})^2 }{n}}$

In [0]:
class NaiveBayes:
    # metoda do obliczenia prawdopodobieństwa (funkcja gaussa)
    @staticmethod
    def gauss(x,mean,std):
        exponent = np.exp(-(x-mean)**2/(2*std**2))
        return 1/(np.sqrt(2*np.pi*std**2))*exponent

    # metoda klasyfikacji
    @staticmethod
    def classify(trainSet, sample):

        # separacja klas z bazy X metodą query wbudowaną w Pandas (można również użyć metod where oraz isin)
        setosa = iris.query('variety == "Setosa"')
        virginica = iris.query('variety == "Virginica"')
        versicolor = iris.query('variety == "Versicolor"')
        classes = [setosa, virginica, versicolor]

        
        probabilities = []
        for c in classes:
            # obliczenie śreniej i odchylenia dla każdego atrybutu dla każdej klasy
            means = []
            stds = []
            for atr in c:
                if c[atr].dtype != 'O':
                    means.append(c[atr].mean())
                    stds.append(c[atr].std())
            
            prob = 1

            # obliczenie składowych prawdopodobieństwa (wykorzystanie gaussa)
            for i in range(4):
                prob *= NaiveBayes.gauss(sample[i],means[i],stds[i])
            probabilities.append(prob)
        
        for i in range(3):
            if probabilities[i] == max(probabilities):
                return classes[i]['variety'].iloc[0]

# 4. <span style='color:green'>Normalizacja, przetasowanie oraz podzielenie danych na część treningową oraz walidacyjną

In [0]:
DataProcessing.normalize(iris)
DataProcessing.shuffle(iris)
trainSet, valuesSet = DataProcessing.split_set(iris)

# 5. <span style='color:green'>Trenowanie modelu oraz testowanie jego precyzji

In [0]:
acc = 0
for i in range(len(valuesSet)):
    sample = valuesSet.iloc[i].to_list()
    if NaiveBayes.classify(trainSet,sample[:4]) == sample[4]:
        acc+=1
print('Uzyskana precyzja dla tego modelu: {}%'.format(int(acc/len(valuesSet)*100)))

Uzyskana precyzja dla tego modelu: 100%


### W żadnym z testowanych modeli precyzja nie spadła ponieżej <span style='color:green'>93</span>%